# Descomposición de Benders
En este notebook haremos una implementación de la descomposición de Benders para resolver nuestro modelo lineal. 

Partiremos instanciando el modelo e importando las librerías y funciones necesarias

In [18]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ipynb.fs.full.Funciones_basicas import *
from ipynb.fs.full.Algoritmos import *
from ipynb.fs.full.Visualizaciones import *
from gurobipy import *
import random

In [19]:
path_geom = '../data/graph_geom_corrected_cycles.csv'
geometry = gpd.read_file(path_geom, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
     
path = '../data/corrected_dijkstra.csv'
csv_grafo = pd.read_csv(path, sep=',')

path_pesos = '../data/pesos_tapas.csv'
pesos = pd.read_csv(path_pesos)

out = {1003950, 15004, 15131, 1003744, 15190, 1003746, 13730, 1003167, 13731, 13732, 13733, 14062, 1003206, 16503, 13735, 16094, 16095}

S = set()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        S.add(origin)
        S.add(dest)

S = list(S)
id_ = {}; _id = {}; l = 0
for u in S:
    id_[u] = l; _id[l] = u
    l += 1

Tree = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        Tree.add_edge(id_[origin], id_[dest])
        
G = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        G.add_edge(id_[origin], id_[dest])
    
N = l

W = [0] * N

for u in range(N):
    if pesos[pesos['ID_tapa'] == _id[u]].shape[0] >= 1:  ## tomamos primera columna con el id, si no hay peso = 0
        W[u] = pesos[pesos['ID_tapa'] == _id[u]].iloc[0]['per_predio']

In [24]:
T = nx.DiGraph()
max_depth = 19
T = Tree.subgraph([node for node in list(Tree.nodes()) if nx.shortest_path_length(Tree, source = node, target = 744) <= max_depth])
K = 8
root = 744
V = list(T.nodes())
N = len(V)
print(f'{N} nodos en el grafo')

142 nodos en el grafo


In [25]:
I = dict()
Path = dict()
for u in V:
    I[u] = get_ideal(T, [0] * 500 * N, u)
    for v in V:
        if v not in I[u]:
            Path[v,u] = list()
        else:
            Path[v,u] = nx.shortest_path(T, source = v, target = u)

In [26]:
model = gurobipy.Model("Muestreo")
x = model.addVars(V, vtype = GRB.BINARY, name ="Nodos raices")
Z = model.addVar(vtype = GRB.CONTINUOUS , obj=1, name ="Ideal de mayor tamano")
y = model.addVars(V, V, vtype = GRB.BINARY, name ="Asignacion de nodo a muestra")


model.addConstr((quicksum(x[i] for i in V) == K), name = "R1")
model.addConstrs((y[j,i] <= x[i] for i in V for j in I[i] ), name = "R2")
model.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
model.addConstrs((x[l] <= 1 - y[j,i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4")
model.addConstrs((y[j,i]  <= y[l, i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4b")
model.addConstrs((quicksum(y[j,i] for j in I[i]) <= Z for i in V), name = "R5")
model.addConstr((x[root] == 1), name = "Root")


model.setObjective(Z, GRB.MINIMIZE)
model.update()
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 43688 rows, 20307 columns and 87373 nonzeros
Model fingerprint: 0x341362d4
Variable types: 1 continuous, 20306 integer (20306 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 39042 rows and 17888 columns
Presolve time: 0.42s
Presolved: 4646 rows, 2419 columns, 29894 nonzeros
Variable types: 0 continuous, 2419 integer (2418 binary)
Found heuristic solution: objective 82.0000000

Root relaxation: objective 3.766980e+00, 4983 iterations, 0.53 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.76698    0 1154   82.00000    3.76698  95.4%     -    1s
H    0     0                      37.0000000    3.76698  89.8%     -    1s
H    0     0                     

## Implementación

In [29]:
master = Model()
master.Params.LazyConstraints = 1

x = master.addVars(V, vtype = GRB.BINARY, name ="x")
theta = master.addVar(name="theta", lb = round(N/K))
R1 = master.addConstr(quicksum(x[i] for i in V) == K, name = "R1")
Root = master.addConstr(x[root] == 1, name = "Root")

master.setObjective(theta, GRB.MINIMIZE)



sub = Model()
sub.Params.OutputFlag = 0
sub.Params.InfUnbdInfo = 1

Z = sub.addVar(vtype = GRB.CONTINUOUS, lb = round(N/K), name ="Z")
y = sub.addVars(V, V, vtype = GRB.CONTINUOUS, lb=0, name ="y")

R2 = sub.addConstrs((y[j,i] <= 0 for i in V for j in I[i] ), name = "R2") # x[i]
R3 = sub.addConstrs((quicksum(y[i,j] for j in Path[i, root]) == 1 for i in V), name = "R3")
R4 = sub.addConstrs((y[j,i] <= 1 for j in V for i in V for l in Path[j,i] if l != i), name = "R4") # -x[l]
R4b = sub.addConstrs((y[j,i]  <= y[l, i] for j in V for i in V for l in Path[j,i] if l != i), name = "R4b")
R5 = sub.addConstrs((quicksum(y[j,i] for j in I[i]) - Z <= 0 for i in V), name = "R5")

sub.setObjective(Z, GRB.MINIMIZE)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


In [30]:
def cutCB(m, where):
    if where == GRB.Callback.MIPSOL:
        
        sol = list()
    #"""          
        for i in V:
            for j in I[i]:
                if m.cbGetSolution(x[i]) > 0.5:
                    R2[i,j].rhs = 1
                    sol.append(i)
                else:
                    R2[i,j].rhs = 0
                                   
            for j in V:
                for l in Path[j,i]:
                    if l != i:
                        if m.cbGetSolution(x[l]) > 0.5:
                            R4[j,i,l].rhs = 0
                        else:
                            R4[j,i,l].rhs = 1
                            

        sample = find_order(T, [0] * 10000, sol, 744)[::-1]
        
        #"""
        V_ = [0]*10000                   
        for sol in sample:
            I_ = get_ideal(T, V_, sol)
            for node in I_:
                y[node, sol].Start = 1
            V_, _ = visit(T, V_, sol)
            
            

        #"""
        sub.optimize()
        

        if sub.Status == GRB.INFEASIBLE:
            #print("Infeasible")
            m.cbLazy(sum(sum(-R2[i,j].farkasDual * x[i] for j in I[i]) for i in V) + sum(- R3[i].farkasDual for i in V) +
                     sum(sum(sum(- R4[j,i,l].farkasDual * (1 - x[l]) for l in Path[j,i] if l != i) for i in V) for j in V) <= 0)

        elif sub.Status == GRB.OPTIMAL: 
            #print("Optimal")
            if sub.objVal > m.cbGetSolution(theta) + 0.001:
                m.cbLazy(sum(sum(R2[i,j].pi * x[i] for j in I[i]) for i in V) + sum(R3[i].pi for i in V) +
                     sum(sum(sum(R4[j,i,l].pi * (1 - x[l]) for l in Path[j,i] if l != i) for i in V) for j in V)  <= theta)


In [31]:
master.optimize(cutCB)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 2 rows, 143 columns and 143 nonzeros
Model fingerprint: 0x0fbda632
Variable types: 1 continuous, 142 integer (142 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 142 columns, 141 nonzeros
Variable types: 1 continuous, 141 integer (141 binary)

Root relaxation: objective 1.800000e+01, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.00000    0    2          -   18.00000      -     -    1s
H    0     0                     115.0000000   18.00000  84.3%     -    2s
     0     0   18.00000    0    2  115.00000   18.00000  84.3%     -    2s
     0     0   18.00000    0    2  1

In [9]:
def find_order(G, V, sample_, s):
    sample = sample_.copy()
    ans = []; Q = deque([]); Q.append(s)
    while Q:
        u = Q.popleft()
        if u in sample:
            ans.append(u)
            sample.remove(u)
        for v in G.predecessors(u):
            if not V[v]:
                Q.append(v)
                V[v] = 1
    return ans

In [10]:
find_order(T, [0]*100000, [744, 2149, 1431, 2166, 422, 628, 1433, 1896], 744)[::-1]

[1896, 1433, 628, 422, 2166, 1431, 2149, 744]